### Написать свой пользовательский dataset используя наследование от torch.utils.data.Dataset

Постановка.  
В общем случае, в задачах ML, мы будем иметь базу данных и файл разметки базы данных (если это обучение с учителем). На пример, если мы решаем задачу детектирования объектов, то в файле разметки для каждого изображения должны содержаться:
1. Название изображения
2. коодринаты рамок (bounding boxes) для каждого объекта на изображении
3. классы каждого изображения

Для задачи MNIST мы имеем 10 папок классов (номера цифр). Так же, имеем файл разметки digitTrain.csv (digitTest.csv). Файлы разметки содержат названия файлов и классы (есть еще колонка - angle наклона цифры, она нам сейчас не нужна). Мы хотим решить задачу классификации. Необходимо написать класс, который наследует от Dataset, на вход принимает файл разметки, на выходе для заданного индекса выдает соответствующую картинку (в формате torch.tensor) и класс изображения (onehot вектор)

In [16]:
import torch
import pandas as pd
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader

In [4]:
df = pd.read_csv('DigitDataset/digitTrain.csv')
df_test = pd.read_csv('DigitDataset/digitTest.csv')
df

,image,digit,angle
0,image2119.png,3,-42
1,image2593.png,3,-15
2,image6703.png,7,-24
3,image6950.png,7,-44
4,image6462.png,7,13
...,...,...,...
4995,image7095.png,8,8
4996,image8253.png,9,39
4997,image7114.png,8,25
4998,image8841.png,9,33


Сделаем то что нам необходимо без использования классов для произвольного idx, например = 2

In [5]:
idx = 2
image = plt.imread('DigitDataset/'+str(df.digit[idx])+'/'+df.image[idx])
image.shape


Необходимо преобразовать данные в тип torch.tensor

In [6]:
image = torch.tensor(image)
print(image.shape)

torch.Size([28, 28])


Мы получили картинку размера 28х28. Torch требует, чтобы формат изображения был CxHxW (канал, высота, ширина)

In [7]:
image = image.unsqueeze(0)
image.shape

torch.Size([1, 28, 28])

Метку класса необходимо преобразовать к onehot вектору

In [19]:
labels = torch.zeros(10)
labels[int(df.digit[idx])] = 1
print(labels,int(df.digit[idx]))

tensor([0., 0., 0., 0., 0., 0., 0., 1., 0., 0.]) 7


Теперь организуем через класс imgDataset

In [8]:

class imgDataset(Dataset):
    def __init__(self,df):
        self.df = df
        
    def __len__(self):
        # your code
        return # количество элементов исходного датасета
    
    def __getitem__(self,idx):
        
        # your code
        
            
        return {"image": image, "label": labels} 
    

dataset = imgDataset(df)
test_dataset = imgDataset(df_test)

Сделайте проверку, того, что выход будет правильным  
Проверка:

In [ ]:
data = next(iter(dataset)) # получили один экземпляр данных (картинка в виде тензора размером 1x28x28 и метка - тензор размером 10)
X = data['image']
Y = data['label']

Dataloader

In [10]:
data_load = DataLoader(dataset,batch_size=32, shuffle=True, pin_memory=True) 
data_load_test = DataLoader(test_dataset,batch_size=len(test_dataset))